## Grover（グローバー）の検索アルゴリズム
グローバーのアルゴリズムは整頓されていないデータの中から目的のデータを探し出すためのアルゴリズムです。理論的には現在の計算機の√Nの計算量で探索ができるということで有名なアルゴリズムです。

## ステップ
全体のアルゴリズムの流れを簡単にまとめます。  
  
１、重ね合わせ状態を作る（すべての量子ビットにHゲートを適用）  
２、マーキングと呼ばれる作業  
３、振幅増幅反転と呼ばれる作業（これでほしい答えを浮かび上がらせる）  

## マーキング
まずは2量子ビットのGroverからです。4通りの組み合わせから11の組み合わせを抜き出してみたいと思います。グローバーのアルゴリズムでは、まず求めたい答えにマーキングを行います。マーキングの方法は簡単で、求めたい解に対応する状態ベクトルだけに-1がかかるようにします。マーキングはゲート操作を使います。

たとえば、00にマーキングしたい場合には、  
[[-1,0,0,0],[0,1,0,0],[0,0,1,0,],[0,0,0,1]] = S[0],S[1],CZ[0,1],S[0],S[1]  
というように、00の状態ベクトル[1,0,0,0]に対応する部分に-1を設定した行列をかけます。  
  
01の時には、  
[[1,0,0,0],[0,-1,0,0],[0,0,1,0,],[0,0,0,1]] = S[1],CZ[0,1],S[1]  
  
10の時には、  
[[1,0,0,0],[0,1,0,0],[0,0,-1,0,],[0,0,0,1]] = S[0],CZ[0,1],S[0]  
  
11の時には、  
[[1,0,0,0],[0,1,0,0],[0,0,1,0,],[0,0,0,-1]] = CZ[0,1]  

こうすることによって対応する行列が実現できます。求めたい解に対するユニタリー変換を作り出す必要があります。まずはBlueqatを使う前にnumpyで確認をしてみます。Sゲート、Hゲート、単位行列Iなどを用意して、

In [1]:
import numpy as np
S = np.array([[1,0],[0,1j]])
H = 1/np.sqrt(2)*np.array([[1,1],[1,-1]])
I = np.eye(2)
CNOT = [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]]

In [2]:
print(np.kron(S,S)@np.kron(I,H)@CNOT@np.kron(I,H)@np.kron(S,S))

[[ 1.+0.00000000e+00j  0.-2.23711432e-17j  0.+0.00000000e+00j
   0.+0.00000000e+00j]
 [ 0.-2.23711432e-17j -1.+0.00000000e+00j  0.+0.00000000e+00j
   0.+0.00000000e+00j]
 [ 0.+0.00000000e+00j  0.+0.00000000e+00j -1.+0.00000000e+00j
   0.+2.23711432e-17j]
 [ 0.+0.00000000e+00j  0.+0.00000000e+00j  0.-2.23711432e-17j
  -1.+0.00000000e+00j]]


In [3]:
print(np.kron(I,S)@np.kron(I,H)@CNOT@np.kron(I,H)@np.kron(I,S)) 

[[ 1.+0.00000000e+00j  0.-2.23711432e-17j  0.+0.00000000e+00j
   0.+0.00000000e+00j]
 [ 0.-2.23711432e-17j -1.+0.00000000e+00j  0.+0.00000000e+00j
   0.+0.00000000e+00j]
 [ 0.+0.00000000e+00j  0.+0.00000000e+00j  1.+0.00000000e+00j
   0.-2.23711432e-17j]
 [ 0.+0.00000000e+00j  0.+0.00000000e+00j  0.+2.23711432e-17j
   1.+0.00000000e+00j]]


In [4]:
print(np.kron(S,I)@np.kron(I,H)@CNOT@np.kron(I,H)@np.kron(S,I))

[[ 1.00000000e+00+0.j -2.23711432e-17+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [-2.23711432e-17+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -1.00000000e+00+0.j
   2.23711432e-17+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -2.23711432e-17+0.j
   1.00000000e+00+0.j]]


In [5]:
print(np.kron(I,H)@CNOT@np.kron(I,H))

[[ 1.00000000e+00 -2.23711432e-17  0.00000000e+00  0.00000000e+00]
 [-2.23711432e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -2.23711432e-17]
 [ 0.00000000e+00  0.00000000e+00  2.23711432e-17 -1.00000000e+00]]


これらは４つのマーキングの行列を表しており、対応する状態ベクトルにマイナスマーキングするようになっています。基本はCNOTゲートをベースに作られています。このような考え方は今後量子ビットが多くなってきたり、より複雑な検索回路を作るのに便利です。

## 振幅増幅反転
そして次のステップは反転させた状態ベクトルの振幅を増幅させ+に戻します。この際に振幅を増幅させるということは、出現確率を増やし求めたい検索結果の出る確率を上げるということに対応します。

振幅増幅反転回路は量子ビット数に依存し、今回のように二量子ビットの場合には、

In [6]:
D = np.array([[-1,1,1,1],[1,-1,1,1],[1,1,-1,1],[1,1,1,-1]])/2

このように、対角項と非対角項の符号が異なるような平均値周りでの反転回路を実現することでできます。このゲートはやはりCNOTをベースに作ることができます。CNOT=CCXにHゲートを適用し、CCZにしてから、XとHゲートで囲むことで実現できます。

In [7]:
X = [[0,1],[1,0]]
print(np.kron(H,H)@np.kron(X,X)@np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])@np.kron(X,X)@np.kron(H,H))

[[ 0.5 -0.5 -0.5 -0.5]
 [-0.5  0.5 -0.5 -0.5]
 [-0.5 -0.5  0.5 -0.5]
 [-0.5 -0.5 -0.5  0.5]]


このように同様の行列ができました。これを適用することで振幅増幅反転を実現できます。

00の回路では、適用してない場合と適用した場合を比較して、

In [8]:
print(np.diag([-1,1,1,1])@np.kron(H,H)@np.array([1,0,0,0]))                                            

[-0.5  0.5  0.5  0.5]


In [9]:
print(D@np.diag([-1,1,1,1])@np.kron(H,H)@np.array([1,0,0,0]))    

[1. 0. 0. 0.]


つぎに01回路は、

In [10]:
print(np.diag([1,-1,1,1])@np.kron(H,H)@np.array([1,0,0,0]))    

[ 0.5 -0.5  0.5  0.5]


In [11]:
print(D@np.diag([1,-1,1,1])@np.kron(H,H)@np.array([1,0,0,0]))    

[0. 1. 0. 0.]


そして10回路、

In [12]:
print(np.diag([1,1,-1,1])@np.kron(H,H)@np.array([1,0,0,0]))

[ 0.5  0.5 -0.5  0.5]


In [13]:
print(D@np.diag([1,1,-1,1])@np.kron(H,H)@np.array([1,0,0,0]))

[0. 0. 1. 0.]


最後に11回路

In [14]:
print(np.diag([1,1,1,-1])@np.kron(H,H)@np.array([1,0,0,0])) 

[ 0.5  0.5  0.5 -0.5]


In [15]:
print(D@np.diag([1,1,1,-1])@np.kron(H,H)@np.array([1,0,0,0])) 

[0. 0. 0. 1.]


このように、振幅増幅反転のユニタリ変換は各パターン共通となっています。こちらをBlueqatに直してみます。インストールがまだの場合には、pipインストールで入手できます。

In [0]:
!pip install blueqat

In [16]:
from blueqat import Circuit
Circuit().h[:].s[:].h[1].cnot[0,1].h[1].s[:].h[:].x[:].h[1].cnot[0,1].h[1].x[:].h[:].run()

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [17]:
Circuit().h[:].s[0].h[1].cnot[0,1].h[1].s[0].h[:].x[:].h[1].cnot[0,1].h[1].x[:].h[:].run()             


array([0.-0.j, 1.+0.j, 0.-0.j, 0.-0.j])

In [18]:
Circuit().h[:].s[1].h[1].cnot[0,1].h[1].s[1].h[:].x[:].h[1].cnot[0,1].h[1].x[:].h[:].run()             


array([0.-0.j, 0.-0.j, 1.+0.j, 0.-0.j])

In [19]:
Circuit().h[:].h[1].cnot[0,1].h[1].h[:].x[:].h[1].cnot[0,1].h[1].x[:].h[:].run()                       


array([0.-0.j, 0.-0.j, 0.-0.j, 1.+0.j])

これがGroverの検索アルゴリズムです。

## 3量子ビットの場合
続いて3量子ビットもやってみます。
CCNOT(CCXもしくはトフォリ)を使います。
まず、マーキングの回路として、8つの状態ベクトルを操作する、np.diag([1,1,1,1,1,1,1,-1])という111だけを抜き出す回路を作ります。これは、CCXの最後のXをHで囲むこと、CCZに変換することで実現できます。

In [20]:
CCNOT = np.array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0]])
print(np.kron(np.kron(I,I),H)@CCNOT@np.kron(np.kron(I,I),H))

[[ 1.00000000e+00 -2.23711432e-17  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.23711432e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -2.23711432e-17
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.23711432e-17  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00 -2.23711432e-17  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23711432e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00 -2.23711432e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   

つづいて、振幅増幅反転もH,XとCCZでつくれます。

In [21]:
print(np.kron(np.kron(H,H),H)@np.kron(np.kron(X,X),X)@np.kron(np.kron(I,I),H)@CCNOT@np.kron(np.kron(I,I),H)@np.kron(np.kron(X,X),X)@np.kron(np.kron(H,H),H))    

[[ 0.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25  0.75 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25  0.75 -0.25 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25  0.75 -0.25 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25  0.75 -0.25 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25  0.75 -0.25 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25  0.75 -0.25]
 [-0.25 -0.25 -0.25 -0.25 -0.25 -0.25 -0.25  0.75]]


Blueqatだと次のようになります。

In [27]:
# 重ね合わせ状態
c = Circuit().h[:]
# マーキング
c.h[2].ccx[0,1,2].h[2]
# 振幅反転増幅
c.h[:].x[:].h[2].ccx[0,1,2].h[2].x[:].h[:]
# 結果 (状態ベクトル)
c.run()

array([0.1767767 +0.00000000e+00j, 0.1767767 -5.02559153e-17j,
       0.1767767 -6.98820710e-17j, 0.1767767 -1.96261557e-17j,
       0.1767767 -6.24500451e-17j, 0.1767767 -7.43202591e-18j,
       0.1767767 -7.43202591e-18j, 0.88388348-2.02214187e-16j])

In [30]:
# 結果 (観測)
c.m[:].run(shots=100)

Counter({'111': 80,
         '100': 5,
         '000': 1,
         '110': 6,
         '011': 3,
         '101': 1,
         '010': 3,
         '001': 1})

高い確率で|111>が得られました。